# Data Collector

This notebook organizes and aggregates data about faculty diversity in gender.

In [14]:
import os
import pandas as pd

The following dictionary maps my custom filename convention to metadata.

In [17]:
DATA_PATH = '../data/excel/'

def getData(fname):
    df = pd.read_excel(DATA_PATH + fname)
    [div, status] = fname.rstrip('.xlsx').split('-')
    df.columns = [
        'Year', 
        '% Women', 
        'Women', 
        'Total', 
        '% Men',
        'Men',
        'Total Duplicate'
    ]
    df.drop(columns=['Total Duplicate'], inplace=True)
    df.set_index('Year', inplace=True)
    df.name = '{}-{}'.format(div, status)
    return df

In [18]:
writer = pd.ExcelWriter('../data/gender.xlsx', engine='xlsxwriter')

for fname in sorted(os.listdir(DATA_PATH)):
    df = getData(fname)
    df.to_excel(writer, sheet_name=df.name, startrow=0, startcol=0)
    
writer.save()